# Camera capture

In [4]:
import cv2
import os

#Comando para lista de dispositivos
#v4l2-ctl --list-devices
# Función para guardar una imagen en la carpeta especificada
def guardar_imagen(frame, folder_path, img_counter):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    img_name = os.path.join(folder_path, f"captura_{img_counter}.png")
    cv2.imwrite(img_name, frame)
    print(f"Imagen guardada: {img_name}")

# Inicialización de captura de video
vid = cv2.VideoCapture(0)

# Contador para el nombre de las imágenes
img_counter = 0

# Carpeta donde se guardarán las imágenes
folder_path = "perfect_choice/cuadro/"
while True: 
    # Capturar frame por frame
    ret, frame = vid.read()
  
    # Mostrar el frame resultante
    cv2.imshow('frame', frame)
      
    # Verificar las teclas presionadas
    key = cv2.waitKey(1) & 0xFF
    
    # Si se presiona la tecla 'q', salir del bucle
    if key == ord('q'):
        break
    
    # Si se presiona la barra espaciadora, guardar la imagen
    if key == ord(' '):
        guardar_imagen(frame, folder_path, img_counter)
        img_counter += 1

# Liberar el objeto de captura y cerrar todas las ventanas OpenCV
vid.release()
cv2.destroyAllWindows()

Imagen guardada: perfect_choice/cuadro/captura_0.png
Imagen guardada: perfect_choice/cuadro/captura_1.png
Imagen guardada: perfect_choice/cuadro/captura_2.png
Imagen guardada: perfect_choice/cuadro/captura_3.png
Imagen guardada: perfect_choice/cuadro/captura_4.png
Imagen guardada: perfect_choice/cuadro/captura_5.png
Imagen guardada: perfect_choice/cuadro/captura_6.png


## Camera Calibration

In [17]:
import cv2
import numpy as np
import os
import glob
 
# Defining the dimensions of checkerboard
CHECKERBOARD = (6,9)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 
# Extracting path of individual image stored in a given directory
images = glob.glob('astra/cuadro/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)


Camera matrix : 

[[488.14038913   0.         319.45549715]
 [  0.         486.90461255 237.52747167]
 [  0.           0.           1.        ]]
dist : 

[[-0.10861338  0.51260148 -0.00627479 -0.00183073 -0.65896048]]
rvecs : 

(array([[-0.43059584],
       [-0.37931028],
       [-1.53375766]]), array([[-0.009669  ],
       [-0.73348296],
       [-1.73000241]]), array([[ 0.02156296],
       [-0.02551099],
       [-1.53768858]]), array([[-0.19363144],
       [ 0.08932663],
       [-1.56623356]]), array([[ 0.1856273 ],
       [-0.47805443],
       [-1.55633692]]), array([[-0.34298687],
       [-0.4090071 ],
       [-1.59125593]]), array([[ 0.22151906],
       [-0.12392557],
       [-1.41615283]]), array([[ 0.25842664],
       [-0.3312131 ],
       [-1.42878996]]), array([[-0.14546338],
       [ 0.00361596],
       [-1.51186433]]), array([[-0.36029751],
       [-0.25883089],
       [-1.4951274 ]]), array([[-0.41978312],
       [ 0.00192161],
       [-1.49769548]]), array([[-0.37449098],
 

In [14]:
fname

'astra/cuadro/captura_9.png'

In [16]:
import cv2 as cv

orbbec_cap = cv.VideoCapture(2, cv.CAP_OBSENSOR)
if orbbec_cap.isOpened() == False:
    exit("Fail to open camera.")

while True:
    if orbbec_cap.grab():
        # RGB data
        ret_bgr, bgr_image = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_BGR_IMAGE)
        # depth data
        ret_depth, depth_map = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_DEPTH_MAP)

        if  ret_bgr:
            color_depth_map = cv.applyColorMap(cv.convertScaleAbs(depth_map, alpha=0.0255), cv.COLORMAP_JET)
            # show the images
            cv.imshow('RBG', bgr_image)
            cv.imshow('Depth', color_depth_map)
        else:
            print("Fail to grab data from the camera.")

        if key == ord('q'):
            break

orbbec_cap.release()

[ERROR:0@2327.172] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@2327.172] global cap.cpp:342 open VIDEOIO(OBSENSOR): backend is generally available but can't be used to capture by index


## Copia imagenes

In [5]:
import os
import random
import shutil

def seleccionar_imagenes(carpeta_origen, carpeta_destino, n):
    # Obtener lista de imágenes en la carpeta origen
    imagenes = [f for f in os.listdir(carpeta_origen) if os.path.isfile(os.path.join(carpeta_origen, f))]
    
    # Verificar si el número de imágenes es mayor que cero
    if len(imagenes) == 0:
        print("No se encontraron imágenes en la carpeta de origen.")
        return
    
    # Seleccionar n números aleatorios en el rango de la cantidad de imágenes
    n = min(n, len(imagenes))
    indices_seleccionados = random.sample(range(len(imagenes)), n)
    
    # Crear la carpeta destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Copiar las imágenes seleccionadas a la carpeta destino
    for i in indices_seleccionados:
        imagen_seleccionada = imagenes[i]
        ruta_origen = os.path.join(carpeta_origen, imagen_seleccionada)
        ruta_destino = os.path.join(carpeta_destino, imagen_seleccionada)
        shutil.copy2(ruta_origen, ruta_destino)
    
    archivo_indices = carpeta_destino + '/indices_seleccionados.txt'    
    # Guardar los índices seleccionados en un archivo .txt
    with open(archivo_indices, 'w') as f:
        for indice in indices_seleccionados:
            f.write(f"{indice}\n")
    print(f"Se copiaron {n} imágenes a la carpeta {carpeta_destino}.")


In [7]:
# Ejemplo de uso
carpeta_origen = '/mnt/6C24E28478939C77/todas'
carpeta_destino = '/mnt/6C24E28478939C77/todas/seleccion'
numero_imagenes_a_seleccionar = 2000

seleccionar_imagenes(carpeta_origen, carpeta_destino, numero_imagenes_a_seleccionar)


Se copiaron 2000 imágenes a la carpeta /mnt/6C24E28478939C77/todas/seleccion.


In [1]:
import serial

In [6]:
ser = serial.Serial('/dev/ttyUSB0')  # open serial port
print(ser.name)         # check which port was really used
ser.write(b'9')     # write a string
ser.close()             # close port

/dev/ttyUSB0


## Capturador serial

In [1]:
import cv2
import os
import serial

ser = serial.Serial('/dev/ttyUSB0')  # open serial port
print(ser.name)         # check which port was really used
grados = 10
# Función para guardar una imagen en la carpeta especificada
def guardar_imagen(frame, folder_path, img_counter):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    img_name = os.path.join(folder_path, f"captura_{img_counter}.png")
    cv2.imwrite(img_name, frame)
    print(f"Imagen guardada: {img_name}")

# Inicialización de captura de video
vid = cv2.VideoCapture(0)
vid2 = cv2.VideoCapture(2)

# Contador para el nombre de las imágenes
img_counter = 0

# Carpeta donde se guardarán las imágenes
folder_path = "prueba/"
while True: 
    # Capturar frame por frame
    ret, frame = vid.read()
    ret2, frame2 = vid2.read()
    # Mostrar el frame resultante
    cv2.imshow('frame', frame)
      
    # Verificar las teclas presionadas
    key = cv2.waitKey(1) & 0xFF
    
    # Si se presiona la tecla 'q', salir del bucle
    if key == ord('q'):
        break
    
    # Si se presiona la barra espaciadora, guardar la imagen
    if key == ord(' '):
        guardar_imagen(frame, folder_path, img_counter)
        guardar_imagen(frame2, folder_path + 'prueba/', img_counter)
        img_counter += 1
        ser.write(grados)     # write a string
        #

# Liberar el objeto de captura y cerrar todas las ventanas OpenCV
vid.release()
cv2.destroyAllWindows()
ser.close()             # close port

/dev/ttyUSB0
Imagen guardada: prueba/captura_0.png
Imagen guardada: prueba/prueba/captura_0.png
Imagen guardada: prueba/captura_1.png
Imagen guardada: prueba/prueba/captura_1.png
Imagen guardada: prueba/captura_2.png
Imagen guardada: prueba/prueba/captura_2.png
Imagen guardada: prueba/captura_3.png
Imagen guardada: prueba/prueba/captura_3.png
Imagen guardada: prueba/captura_4.png
Imagen guardada: prueba/prueba/captura_4.png
Imagen guardada: prueba/captura_5.png
Imagen guardada: prueba/prueba/captura_5.png
